## Correcting local biases in sampling

In [1]:
import scipy
import numpy as np
import pandas as pd

import collections

def recursively_default_dict():
        return collections.defaultdict(recursively_default_dict)

from sklearn.neighbors import KernelDensity
from sklearn.decomposition import PCA
from sklearn.model_selection import GridSearchCV
from sklearn.cluster import MeanShift, estimate_bandwidth
from sklearn.metrics.pairwise import pairwise_distances
from sklearn.preprocessing import scale

from scipy.stats.stats import pearsonr 

from scipy.stats import invgamma 
from scipy.stats import beta
import matplotlib.pyplot as plt

import plotly
import plotly.plotly as py
import plotly.graph_objs as go
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
from plotly.graph_objs import *

init_notebook_mode(connected=True)

In our first post we examined how to use frequency vectors to generate haplotypes and populations. We then proceeded to generate a universe of frequency vectors, whose distance in feature space allowed us to chose the relative differentiation of the populations we would simulate.

What i didn't touch on in that post was the importance of sampling in principal component analysis. In the last section, i chose vectors close to one another, together with vectors far distant, in order to produce differentiated populations. If you tweeked the population sizes, you might have noticed that if some of the close together populations largely outweighed the rest, the distances to the more differentiated clusters would be reduced.

- see [McVean 2009](http://journals.plos.org/plosgenetics/article?id=10.1371/journal.pgen.1000686) for a study case.


## Index

### I. Generating Frequency Vectors

### II. A One-Shot example

#### a. Biased sampling

#### b. MS correction

#### c. Unbiased sampling

### III. Permutations.




### I. Generating Frequency Vectors.

We will start by generating a space of vectors. 

For each vector we will extract _L_ random samples from the **beta** distribution. We will repeat this process as we vary the mean and variance (parameters _a_ and _b_) of the distribution. The ranges along which to vary these, the number of steps and the number of vectors to generate from each combination of _a_ and _b_ can be specified at the beginning of the next block of code.

In [2]:
# Simulate frequency vectors. 
# We must first define the number of populations, the length of the haplotypes desired, and their respective population sizes
L= 150

import itertools as it
n= 15

# Vary a (beta distribution parameter).
a_range= np.linspace(1,2,15)
a_set= [i for i in a_range for _ in range(n)]

# vary b.
b_range= np.linspace(0.1,.4,15)
b_set= [i for i in b_range for _ in range(n)]

## length of haplotypes to extract.
L_set= [L] * n * 15


background= np.array([a_set,b_set,L_set]).T

vector_lib= []
for k in range(background.shape[0]):
    
    probs= beta.rvs(background[k,0], background[k,1], size=int(background[k,2]))
    probs[(probs > 1)]= 1
    
    
    vector_lib.append(probs)

vector_lib= np.array(vector_lib)

In [149]:
print('Number of frequency vectors of size {} generated: {}'.format(vector_lib.shape[1],vector_lib.shape[0]))


Number of frequency vectors of size 150 generated: 225


Perform PCA on data set of frequency vectors created.

This allows us to:

i) verify the desired randomness of the process of generating vectors.
 
ii) select vectors based on how much they resemble each other. 

We have verified in another post how distance in the feature space created this way relates positively to genetic distance as measured by Fst (see **10. A link to Fsts**).


In [150]:
## PCA on vectors simulated
n_comp = 3

pca = PCA(n_components=n_comp, whiten=False,svd_solver='randomized').fit(vector_lib)
features = pca.transform(vector_lib) * pca.explained_variance_ratio_

print("; ".join(['PC{0}: {1}'.format(x+1,round(pca.explained_variance_ratio_[x],3)) for x in range(n_comp)]))
print('features shape: {}'.format(features.shape))

PC1: 0.025; PC2: 0.022; PC3: 0.02
features shape: (225, 3)


In [151]:
## Plot vector PCA
fig_data= [go.Scatter3d(
        x = features[:,0],
        y = features[:,1],
        z = features[:,2],
        type='scatter3d',
        mode= "markers",
        text= ['a: {}; b: {}, L: {}; index = {}'.format(background[k,0],background[k,1],background[k,2], k) for k in range(background.shape[0])],
        marker= {
        'line': {'width': 0},
        'size': 4,
        'symbol': 'circle',
      "opacity": .8
      }
    )]


layout = go.Layout(
    margin=dict(
        l=0,
        r=0,
        b=0,
        t=0
    )
)

fig = go.Figure(data=fig_data, layout=layout)
iplot(fig)


 **Fig. 1** PCA on frequency vectors generated from the beta distribution. Parameters a and b were made to vary between 
1-2 and .1-.4 respectively at steps of .1. For each combination of parameters 15 vectors were produced.


The point of this is to peruse frequency vector space, and chose populations. Because PCA distances equal correlation between observations, the closer the points the higher the Fst between the pops selected.

### II. One-Shot example.

#### II a. Biased sampling of frequency vectors

chose populations and bias sizes below.

Haplotypes for each population will be generated using the frequency vectors as the probabilities of alleles.

In [245]:
### Select frequency vectors and draw haplotypes.
## Pops selected by Indicies.
Pops= [80,148,109,17]
N_pops= len(Pops)

## Population Sizes and labels
Sizes_bias= [130,80,300,35]
labels_bias= np.repeat(np.array([x for x in range(N_pops)]),Sizes_bias)

## Number of pops

data_ex= []

for k in range(N_pops):
    
    probs= vector_lib[Pops[k],:]
    
    m= Sizes_bias[k]
    Haps= [[np.random.choice([1,0],p= [1-probs[x],probs[x]]) for x in range(L)] for acc in range(m)]
    
    data_ex.extend(Haps)

data_ex= np.array(data_ex)
print(data_ex.shape)

#data_ex= scale(data_ex)

(545, 150)


We will also calculate genetic distances between pairs of individuals based on the haplotypes. Calculation will be simple, distance based (n_diffs / hap_length).

In [246]:
### Calculate individual pairwise distances for biased sampling.
def pairwise_gen(x,y):
    miss= 0
    same= 0
    if len(x) != len(y):
        return 'vector lengths differ'
    else:
        for n in range(len(x)):
            if x[n] == y[n]:
                same += 1
        return 1 - same / (len(x) - miss)

bias_gen_diffs= pairwise_distances(data_ex,metric= pairwise_gen)
bias_gen_diffs= np.array(bias_gen_diffs)

iugen= np.triu_indices(bias_gen_diffs.shape[0],1)
bias_gen_diffs= bias_gen_diffs[iugen]

From the frequency vectors selected we can also calculate pairwise Fst's.

These will be used later to compare to pairswise centroid distances in feature space.

In [247]:
### Calculate pairwise Fst based on frequency vectors selected.

def return_fsts2(freq_array):
    pops= range(freq_array.shape[0])
    H= {pop: [1-(freq_array[pop,x]**2 + (1 - freq_array[pop,x])**2) for x in range(freq_array.shape[1])] for pop in range(freq_array.shape[0])}
    Store= []

    for comb in it.combinations(H.keys(),2):
        P= [sum([freq_array[x,i] for x in comb]) / len(comb) for i in range(freq_array.shape[1])]
        HT= [2 * P[x] * (1 - P[x]) for x in range(len(P))]
        per_locus_fst= [[(HT[x] - np.mean([H[p][x] for p in comb])) / HT[x],0][int(HT[x] == 0)] for x in range(len(P))]
        per_locus_fst= np.nan_to_num(per_locus_fst)
        Fst= np.mean(per_locus_fst)

        Store.append([comb,Fst])
    
    
    ### total fst:
    P= [sum([freq_array[x,i] for x in pops]) / len(pops) for i in range(freq_array.shape[1])]
    HT= [2 * P[x] * (1 - P[x]) for x in range(len(P))]
    FST= np.mean([(HT[x] - np.mean([H[p][x] for p in pops])) / HT[x] for x in range(len(P))])
    
    return pd.DataFrame(Store,columns= ['pops','fst'])

freqs_selected= vector_lib[Pops,:]
Pairwise= return_fsts2(freqs_selected)

fsts_compare = scale(Pairwise.fst)
Pairwise

,pops,fst
0,"(0, 1)",0.113636
1,"(0, 2)",0.100276
2,"(0, 3)",0.132882
3,"(1, 2)",0.100213
4,"(1, 3)",0.116162
5,"(2, 3)",0.103910


Finally we perform PCA on the haplotypes generated and keep the first 5 PCs. Notice that right now the multiplication by eigenvalues is commented out. 

To run this test with the multiplication by eigenvalues you can uncomment that (i'll write something simpler later on).

In [248]:
### PCA on haplotypes drawn.
n_comp = 5

pca = PCA(n_components=n_comp, whiten=False,svd_solver='randomized').fit(data_ex)

bias_features= pca.transform(data_ex)# * pca.explained_variance_ratio_

var_comps= pca.explained_variance_ratio_
print("; ".join(['PC{0}: {1}'.format(x+1,round(var_comps[x],3)) for x in range(n_comp)]))
print(bias_features.shape)

PC1: 0.131; PC2: 0.08; PC3: 0.039; PC4: 0.019; PC5: 0.018
(545, 5)


Estimating population centroids in feature space and plot the PCA on biased-haplotype data set

In [249]:
## Calculate centroids
bias_centroids= [np.mean(bias_features[[y for y in range(bias_features.shape[0]) if labels_bias[y] == z],:],axis= 0) for z in range(N_pops)]
bias_centroids= np.array(bias_centroids)

## plot
fig_data= [go.Scatter(
        x = bias_features[[x for x in range(sum(Sizes_bias)) if labels_bias[x] == i],0],
        y = bias_features[[x for x in range(sum(Sizes_bias)) if labels_bias[x] == i],1],       
        type='scatter',
        mode= "markers",
        marker= {
        'line': {'width': 0},
        'size': 8,
        'symbol': 'circle',
      "opacity": .8
      },
      name= str(i)
    ) for i in range(N_pops)]


fig_data.append(
    go.Scatter(
        x= bias_centroids[:,0],
        y= bias_centroids[:,1],
        type= 'scatter',
        mode= 'markers',
        name= 'centres',
        marker= {
        'line': {'width': 1},
        'size': 10,
        'symbol': 'cross'
        }
    )
)

layout = go.Layout(
    title= 'Biased sampling; eigenvalues factored in',
    yaxis=dict(
        title='PC2: {}'.format(round(var_comps[1],3))),
    xaxis=dict(
    title= 'PC1: {}'.format(round(var_comps[0],3))),
)


fig = go.Figure(data=fig_data, layout=layout)
iplot(fig)

**Fig. 2** PCA on haplotypes sampled unevenly from the frequency vectors selected above. 

Calculate Pairwise centroid distances in feature space (remember we kept 5 components), and individual pairwise distances in this space also.

(4, 5)

In [301]:
## centroid distances global
iu1= np.triu_indices(N_pops,1)
bias_pair_dist= pairwise_distances(bias_centroids,metric= 'euclidean')
bias_pair_dist= bias_pair_dist[iu1]
#bias_pair_dist= scale(bias_pair_dist)

### centroid distances by PC:
dist_PC_bias= {}
for PC in range(bias_centroids.shape[1]):
    bias_PC_dist= pairwise_distances(bias_centroids[:,PC].reshape(-1,1),metric= 'euclidean')
    bias_PC_dist= bias_PC_dist[iu1]
    #bias_pair_dist= scale(bias_pair_dist)
    dist_PC_bias[PC]= bias_PC_dist
    
## Individual distances:
bias_feat_dist= pairwise_distances(bias_features, metric= 'euclidean')
bias_feat_dist= bias_feat_dist[iugen]
bias_feat_dist= scale(bias_feat_dist)


### Pearson's r between individual pairwise genetic and feature space distances.
bias_feat_Pearson= pearsonr(bias_feat_dist,bias_gen_diffs)
print('Our first result: Pearson r between individual feature space and genetic distances: {}'.format(bias_feat_Pearson[0]))


Our first result: Pearson r between individual feature space and genetic distances: 0.8266530362800438


In [302]:
bias_pair_dist

array([ 3.56184958,  3.34561412,  3.85166481,  3.32164466,  3.45954717,
        3.26572654])

I chose the first two populations to play the outliers, the rest to be a close pack. To two of these i gave population sizes of 300 and 180, six and 3.6 times the size of the largest outlying population. 

The distortion can be seen in that our outlying populations don't appear as far as we would have expected them to given their vectors alone. They tend to appear in the center because of their reduced impact on variance components.

As remarked by McVean, this can be a problem when deriving conclusions from relative distances in feature space.


#### II. b. MeanShift correction.

MeanShift allows us to identify clusters in feature space, we will resample those clusters equally, inverse transform their coordinates and perform the PCA anew. The actual data is transposed onto the resulting space.

In [303]:
def local_sampling_correct(data_now,n_comp):
    pca = PCA(n_components=n_comp, whiten=False,svd_solver='randomized').fit(data_now)
    feats= pca.transform(data_now)
    
    N= 50
    bandwidth = estimate_bandwidth(feats, quantile=0.2)
    params = {'bandwidth': np.linspace(np.min(feats), np.max(feats),30)}
    grid = GridSearchCV(KernelDensity(algorithm = "ball_tree",breadth_first = False), params,verbose=0)
    
    ## perform MeanShift clustering.
    ms = MeanShift(bandwidth=bandwidth, bin_seeding=True, cluster_all=False, min_bin_freq=5)
    ms.fit(feats)
    labels1 = ms.labels_
    label_select = {y:[x for x in range(len(labels1)) if labels1[x] == y] for y in sorted(list(set(labels1))) if y != -1}

    ## Extract the KDE of each cluster identified by MS.
    Proxy_data= []

    for lab in label_select.keys():
        if len(label_select[lab]) < 3:
            continue
            
        Quanted_set= feats[label_select[lab],:]
        grid.fit(Quanted_set)

        kde = grid.best_estimator_
        Extract= kde.sample(N)
        Return= pca.inverse_transform(Extract)
        
        #Return= data_now[np.random.choice(label_select[lab],N),:]
        Proxy_data.extend(Return)
    
    Proxy_data= np.array(Proxy_data)
    
    print([len(x) for x in label_select.values()])
    pca2 = PCA(n_components=n_comp, whiten=False,svd_solver='randomized').fit(Proxy_data)
    var_comp= pca2.explained_variance_ratio_
    
    New_features= pca2.transform(data_now)# * var_comp
    return New_features, var_comp


New_features,var_comp= local_sampling_correct(data_ex,n_comp)

[293, 130, 80, 35]


Plotting our original samples onto our re-computed feature space:

In [304]:
corr_centroids= [np.mean(New_features[[y for y in range(New_features.shape[0]) if labels_bias[y] == z],:],axis= 0) for z in range(N_pops)]
corr_centroids= np.array(corr_centroids)

fig_data= [go.Scatter(
        x = New_features[[x for x in range(sum(Sizes_bias)) if labels_bias[x] == i],0],
        y = New_features[[x for x in range(sum(Sizes_bias)) if labels_bias[x] == i],1],
        type='scatter',
        mode= "markers",
        marker= {
        'line': {'width': 0},
        'size': 8,
        'symbol': 'circle',
      "opacity": .8
      },
      name= str(i)
    ) for i in range(N_pops)]

fig_data.append(
    go.Scatter(
        x= corr_centroids[:,0],
        y= corr_centroids[:,1],
        type= 'scatter',
        mode= 'markers',
        name= 'centres',
        marker= {
        'line': {'width': 1},
        'size': 10,
        'symbol': 'cross'
        }
    )
)


layout = go.Layout(
    title= 'Biased corrected, eigenvalues not factored in',
    yaxis=dict(
        title='PC2: {}'.format(round(var_comp[1],3))),
    xaxis=dict(
    title= 'PC1: {}'.format(round(var_comp[0],3))),
)

fig = go.Figure(data=fig_data, layout=layout)
iplot(fig)

**Fig. 3** MS corrected PCA of unvenly sampled haplotypes. MS clustering was first applied following an initial PCA of the biased data set. The KDE of each cluster identified this way was used to resample equally from that distribution. 50 observations were resampled from the distribution of each cluster identified by MS. Each observation from the resulting data set was inverse transformed and a new PCA was conducted on this data. Finally, the original haplotypes were projected onto the new feature space.

Very different from the original PCA on the biased data set.

We now calculate the distances between centroids in this feature space and the pairwise individual distances, compare the latter to genetic distances.

In [305]:
corr_centroids.shape

(4, 5)

In [306]:
### Centroid distances global
iu1= np.triu_indices(N_pops,1)
corrected_pair_dist= pairwise_distances(corr_centroids,metric= 'euclidean')
corrected_pair_dist= corrected_pair_dist[iu1]
#corrected_pair_dist= scale(corrected_pair_dist)

### centroid distances by PC
dist_PC_corrected= {}
for PC in range(corr_centroids.shape[1]):
    corrected_PC_dist= pairwise_distances(corr_centroids[:,PC].reshape(-1,1),metric= 'euclidean')
    corrected_PC_dist= corrected_PC_dist[iu1]
    #corrected_pair_dist= scale(corrected_pair_dist)
    dist_PC_corrected[PC]= corrected_PC_dist

## Individual distances:
corrected_feat_dist= pairwise_distances(New_features, metric= 'euclidean')
corrected_feat_dist= corrected_feat_dist[iugen]
corrected_feat_dist= scale(corrected_feat_dist)

corrected_gen_pearson= pearsonr(corrected_feat_dist,bias_gen_diffs)

print('Pearon r of individual genetic distances versus feature space distances following correction: {}'.format(round(corrected_gen_pearson[0],3)))


Pearon r of individual genetic distances versus feature space distances following correction: 0.827


In [307]:
corrected_pair_dist

array([ 3.56184444,  3.34561439,  3.85165321,  3.32165008,  3.45949017,
        3.26571004])

#### II. c. Even sampling.


We can compare this output with what we would have gotten from sampling equally across our selected vectors.

For this purpose we sample equally from the same frequency vectors as in the biased scenario and perform PCA on the resulting data set.

In [308]:
#### Selecting new, equal sample sizes but derive haplotypes from the same frequency vectors.

Sizes= [50,50,50,50]
labels= np.repeat(np.array([x for x in range(N_pops)]),Sizes)

data= []

for k in range(N_pops):
    
    probs= vector_lib[Pops[k],:]
    
    m= Sizes[k]
    Haps= [[np.random.choice([1,0],p= [1-probs[x],probs[x]]) for x in range(L)] for acc in range(m)]
    
    data.extend(Haps)

data= np.array(data)
#data= scale(data)

#### clalculate pairwise genetic distances
iugen_unbiased= np.triu_indices(data.shape[0],1)

unbias_gen_diffs= pairwise_distances(data,metric= pairwise_gen)
unbias_gen_diffs= np.array(unbias_gen_diffs)

unbias_gen_diffs= unbias_gen_diffs[iugen_unbiased]

### perform PCA

n_comp = 5

pca = PCA(n_components=n_comp, whiten=False,svd_solver='randomized').fit(data)

features= pca.transform(data)# * pca.explained_variance_ratio_

var_comps= pca.explained_variance_ratio_
print("; ".join(['PC{0}: {1}'.format(x+1,round(var_comps[x],3)) for x in range(n_comp)]))
print(features.shape)


#### Calculate centroids of labelled data in feature space.
unbias_centroids= [np.mean(features[[y for y in range(features.shape[0]) if labels[y] == z],:],axis= 0) for z in range(N_pops)]
unbias_centroids= np.array(unbias_centroids)


#### Plot projections + Centroids.

fig_data= [go.Scatter(
        x = features[[x for x in range(sum(Sizes)) if labels[x] == i],0],
        y = features[[x for x in range(sum(Sizes)) if labels[x] == i],1],
        type='scatter',
        mode= "markers",
        marker= {
        'line': {'width': 0},
        'size': 8,
        'symbol': 'circle',
      "opacity": .8
      },
      name= str(i)
    ) for i in range(N_pops)]


fig_data.append(
    go.Scatter(
        x= unbias_centroids[:,0],
        y= unbias_centroids[:,1],
        type= 'scatter',
        mode= 'markers',
        name= 'centres',
        marker= {
        'line': {'width': 1},
        'size': 10,
        'symbol': 'cross'
        }
    )
)

layout = go.Layout(
    title= 'Unbiased sampling; eigenvalues factored in',
    yaxis=dict(
        title='PC2: {}'.format(round(var_comps[1],3))),
    xaxis=dict(
    title= 'PC1: {}'.format(round(var_comps[0],3))),
)

fig = go.Figure(data=fig_data, layout=layout)
iplot(fig)

PC1: 0.14; PC2: 0.101; PC3: 0.087; PC4: 0.021; PC5: 0.019
(200, 5)


**Fig. 4** PCA on evenly sampled haplotypes from the same frequency vectors as above. 50 haplotypes generated from each vector.

Calculate pairwise distances. compare to genetic distances.

In [309]:
### centroid distances global
unbias_pair_dist= pairwise_distances(unbias_centroids,metric= 'euclidean')
unbias_pair_dist= unbias_pair_dist[iu1]
#unbias_pair_dist= scale(unbias_pair_dist)

### centroid distances by PC
dist_PC_even= {}
for PC in range(unbias_centroids.shape[1]):
    unbias_PC_dist= pairwise_distances(unbias_centroids[:,PC].reshape(-1,1),metric= 'euclidean')
    unbias_PC_dist= unbias_PC_dist[iu1]
    #unbias_pair_dist= scale(unbias_pair_dist)
    dist_PC_even[PC]= unbias_PC_dist

## Individual distances:
unbiased_feat_dist= pairwise_distances(features, metric= 'euclidean')

unbiased_feat_dist= unbiased_feat_dist[iugen_unbiased]
unbiased_feat_dist= scale(unbiased_feat_dist)

unbiased_gen_pearson= pearsonr(unbiased_feat_dist,unbias_gen_diffs)

print('Pearson r on individual genetic and feature space distances in the unbiased sampling scenario: {}'.format(round(corrected_gen_pearson[0])))


Pearson r on individual genetic and feature space distances in the unbiased sampling scenario: 1.0


In [312]:
fig_data= [go.Scatter(
    x= dist_PC_even[PC],
    y= dist_PC_corrected[PC],
    mode= 'markers',
    marker= dict(
        color= PC,
        opacity= .6
    ),
    name= 'PC: {}, {}'.format(PC,round(pearsonr(dist_PC_even[PC],dist_PC_corrected[PC])[0],2))
    ) for PC in dist_PC_even.keys()
]

layout = go.Layout(
    title= 'MS correction distances',
    yaxis=dict(
        title='biased and corrected distances'),
    xaxis=dict(
        title='unbiased distances')
)

fig= go.Figure(data=fig_data, layout=layout)
iplot(fig)

**Fig. 5** Principal-component-wise relation between pairwise centroid distances in Unbiased versus Corrected scenarios. Average distances were calculated between the projections of population specific haplotypes along specific PCs. Pearson's r was calculated using the function `pearsonr` of the python package `scipy.stats.stats`.

In [313]:
t= np.array([
    unbias_pair_dist,
    bias_pair_dist,
    corrected_pair_dist
]).T

fig_data= [go.Scatter(
    x= t[:,0],
    y= t[:,i],
    mode= 'markers',
    marker= dict(
        color= i,
        opacity= .6
    ),
    name= ['bias','corrected'][i-1]
    ) for i in [1,2]
]

layout = go.Layout(
    title= 'MS correction distances',
    yaxis=dict(
        title='biased and corrected distances'),
    xaxis=dict(
        title='unbiased distances')
)

fig= go.Figure(data=fig_data, layout=layout)
iplot(fig)

**Fig. 6** Relation between pairwise centroid distances in unbiased scenario versus biased and corrected scenarios. Distances were left unscaled.

In [314]:

fig_fsts= [go.Scatter(
    x= fsts_compare,
    y= t[:,i],
    mode= 'markers',
    marker= dict(
        color= i,
        opacity= .6
    ),
    name= ['unbiased','biased','corrected'][i-1]
    ) for i in [0,1,2]
]

layout = go.Layout(
    title= 'PCA to genetic distances',
    yaxis=dict(
        title='centrois distances in feature space'),
    xaxis=dict(
        title='normalized Fst')
)

fig= go.Figure(data=fig_fsts, layout=layout)
iplot(fig)

**Fig. 7** Relationship between Fst and feature space distances in the three scenarios considered: unbiased, biased and MS corrected.


### II. Permutations.

We will now repeat this process sequentially, to get an idea of how much this method actually corrects distances between pops.

At each repetition we will choose a fixed number of frequency vectors from the _Vector Universe_ created at the top of this page. We then perform a biased and an unbiased sampling of each, and perform PCA on both. For each scenario we calculate the pairwise eucledian distances between the centroids of populations and normalize them. 

We then apply the MScorrection to the feature space of the biased scenario and recalculate pairwise centroid distances and normalize them.

This will allow us to compare the unbiased distances to biased and corrected distances. Hopefully, we will have reduced the distortion produced by the biases in sampling.


- You can choose to to multiply PCs of PCAs performed by their respective eigenvalues by typing `Eigen = True` below.

- You can choose to scale Haplotype matrices produced by feature by typing `Scale = True` below.

In [340]:
### Select frequency vectors and draw haplotypes.
Eigen = False
Scale= False

length_range= [150,300]
length_step= 10

N_pops= 4 # Number of pops

n_comp= 5 # components to keep following PCA

Iter= 20 # repeats

N_sims= 100 # number of haplotypes to generate from each pop in the unbiased scenario.


## Population Sizes and labels
bias_scheme= np.random.choice(range(25,200),N_pops,replace= False)
unbiased_sheme= np.repeat(N_sims,N_pops)

bias_labels= np.repeat(np.array([x for x in range(N_pops)]),bias_scheme)
unbias_labels= np.repeat(np.array([x for x in range(N_pops)]),unbiased_sheme)

### store distances between centroids
biased_pairwise= []
unbiased_pairwise= []
corrected_pairwise= []

### store PC projection:
dist_PC_even= {x:[] for x in range(n_comp)}
dist_PC_bias= {x:[] for x in range(n_comp)}
dist_PC_corrected= {x:[] for x in range(n_comp)}


### store fsts
fst_store= []

### store Pearson's r comparing gen_diffs and feature space diffs across scenarios
biased_pears= []
corrected_pears= []
unbiased_pears= []

### triangular matrices extract.
iu1= np.triu_indices(N_pops,1) # for centroid comparison

iu_unbias= np.triu_indices(sum(unbiased_sheme),1)
iu_bias= np.triu_indices(sum(bias_scheme),1)

### proceed.

for rep in range(Iter):
    Pops= np.random.choice(vector_lib.shape[0],N_pops,replace= False)
    print('vectors selected: {}'.format(Pops))
    ########## FST
    
    freqs_selected= vector_lib[Pops,:]
    Pairwise= return_fsts2(freqs_selected)

    #fsts_compare = scale(Pairwise.fst)
    fsts_compare= Pairwise.fst
    fst_store.extend(fsts_compare)
    #########################################################
    ########### PCA ####################################
    #########################################################
    ############# unbiased sample
    
    #### generate data and perform PCA.
    data= []

    for k in range(N_pops):

        probs= vector_lib[Pops[k],:]
        
        m= unbiased_sheme[k]
        Haps= [[np.random.choice([1,0],p= [1-probs[x],probs[x]]) for x in range(L)] for acc in range(m)]

        data.extend(Haps)
    
    data1= np.array(data)
    
    if Scale:
        data1= scale(data1)
    
    pca = PCA(n_components=n_comp, whiten=False,svd_solver='randomized').fit(data1)
    
    feat_unbias= pca.transform(data1)
    
    if Eigen:
        feat_unbias= feat_unbias * pca.explained_variance_ratio_
    
    unbias_proj.append(np.concatenate(feat_unbias[:,:3]))
    
    ####### centroid comparison
    unbias_centroids= [np.mean(feat_unbias[[y for y in range(feat_unbias.shape[0]) if unbias_labels[y] == z],:],axis= 0) for z in range(N_pops)]
    unbias_centroids= np.array(unbias_centroids)
    
    unbias_pair_dist= pairwise_distances(unbias_centroids,metric= 'euclidean')
    unbias_pair_dist= unbias_pair_dist[iu1]
    
    #unbias_pair_dist= scale(unbias_pair_dist)
    unbiased_pairwise.extend(unbias_pair_dist)
    
    ######## ind distances
    ### centroid distances by PC
    
    for PC in range(unbias_centroids.shape[1]):
        unbias_PC_dist= pairwise_distances(unbias_centroids[:,PC].reshape(-1,1),metric= 'euclidean')
        unbias_PC_dist= unbias_PC_dist[iu1]
        unbias_PC_dist= scale(unbias_PC_dist)
        dist_PC_even[PC].extend(unbias_PC_dist)
    
    #################################################
    ############## biased sample
    
    #### generate data and perform PCA
    data= []

    for k in range(N_pops):

        probs= vector_lib[Pops[k],:]

        m= bias_scheme[k]
        Haps= [[np.random.choice([1,0],p= [1-probs[x],probs[x]]) for x in range(L)] for acc in range(m)]

        data.extend(Haps)

    data2= np.array(data)
    
    if Scale:
        data2= scale(data2)
        
    pca = PCA(n_components=n_comp, whiten=False,svd_solver='randomized').fit(data2)
    
    feat_bias= pca.transform(data2)
    
    if Eigen:
        feat_bias= feat_bias * pca.explained_variance_ratio_
    
    bias_proj.append(np.concatenate(feat_bias[:,:3]))
    #### Centroid distances
    bias_centroids= [np.mean(feat_bias[[y for y in range(feat_bias.shape[0]) if bias_labels[y] == z],:],axis= 0) for z in range(N_pops)]
    bias_centroids= np.array(bias_centroids)
    
    bias_pair_dist= pairwise_distances(bias_centroids,metric= 'euclidean')
    bias_pair_dist= bias_pair_dist[iu1]
    #bias_pair_dist= scale(bias_pair_dist)
    biased_pairwise.extend(bias_pair_dist)

    ########
    ### genetic data
    for PC in range(bias_centroids.shape[1]):
        bias_PC_dist= pairwise_distances(bias_centroids[:,PC].reshape(-1,1),metric= 'euclidean')
        bias_PC_dist= bias_PC_dist[iu1]
        bias_PC_dist= scale(bias_PC_dist)
        dist_PC_bias[PC].extend(bias_PC_dist)

    ###############################################################"
    ################## bias correct
    ### perform MS correction on biased samples
    feat_correct,var_comp= local_sampling_correct(data2,n_comp)
    
    corrected_proj.append(np.concatenate(feat_correct[:,:3]))
    ### centroid Distances
    centroids= [np.mean(feat_correct[[y for y in range(feat_correct.shape[0]) if bias_labels[y] == z],:],axis= 0) for z in range(N_pops)]
    centroids= np.array(centroids)
    pair_dist= pairwise_distances(centroids,metric= 'euclidean')
    pair_dist= pair_dist[iu1]
    #pair_dist= scale(pair_dist)
    corrected_pairwise.extend(pair_dist)
    
    ### PC-wise centroid comparison
    for PC in range(centroids.shape[1]):
        corrected_PC_dist= pairwise_distances(centroids[:,PC].reshape(-1,1),metric= 'euclidean')
        corrected_PC_dist= corrected_PC_dist[iu1]
        corrected_PC_dist= scale(corrected_PC_dist)
        dist_PC_corrected[PC].extend(corrected_PC_dist)
    
    
    t= np.array([
    fsts_compare,
    unbias_pair_dist,
    bias_pair_dist,
    pair_dist
    ]).T



vectors selected: [193  89   1  34]
[147, 108, 96, 36]
vectors selected: [151 128  31 177]
[161, 107, 91, 36]
vectors selected: [ 75 182 198 171]
[166, 106, 91, 36]
vectors selected: [ 28 179  62  58]
[165, 103, 96, 36]
vectors selected: [  8 101 113  97]
[163, 108, 92, 36]
vectors selected: [168  31 109 141]
[149, 108, 96, 36]
vectors selected: [147 120 154   6]
[152, 107, 96, 36]
vectors selected: [142 160 138 155]
[162, 108, 96, 36]
vectors selected: [119  97 113  61]
[151, 109, 96, 36]
vectors selected: [210  72  38 219]
[162, 109, 95, 36]
vectors selected: [133 218 153  44]
[161, 105, 96, 36]
vectors selected: [128 177 164 201]
[160, 107, 96, 36]
vectors selected: [163 177 161 218]
[166, 107, 94, 36]
vectors selected: [155 154 186  65]
[163, 105, 96, 36]
vectors selected: [ 62  36 115  75]
[165, 109, 95, 36]
vectors selected: [ 85 182  92 167]
[166, 106, 93, 36]
vectors selected: [ 33  98 157  52]
[161, 108, 96, 36]
vectors selected: [178 160 213 168]
[159, 108, 94, 36]
vectors se

In [352]:
fig_data= [go.Scatter(
    x= dist_PC_even[PC],
    y= dist_PC_corrected[PC],
    mode= 'markers',
    marker= dict(
        color= PC,
        opacity= .6
    ),
    name= 'PC: {}, {}'.format(PC,round(pearsonr(dist_PC_even[PC],dist_PC_corrected[PC])[0],2))
    ) for PC in [0,1,2]
]

layout = go.Layout(
    title= 'MS correction distances',
    yaxis=dict(
        title='biased and corrected distances'),
    xaxis=dict(
        title='unbiased distances')
)

fig= go.Figure(data=fig_data, layout=layout)
iplot(fig)

**Fig. 8** Relation between pairwise centroid distances in unbiased scenario versus biased and corrected scenarios across simulations. Distances were left unscaled.

In [335]:
t= np.array([
    unbiased_pairwise,
    biased_pairwise,
    corrected_pairwise
]).T

t2= np.array([
    unbias_proj,
    bias_proj,
    corrected_proj
]).T

In [336]:
pearsons= [pearsonr(fst_store,t[:,x])[0] for x in range(t.shape[1])]

fig_data= [go.Scatter(
    x= fst_store,
    y= t[:,i],
    mode= 'markers',
    marker= dict(
        color= i,
        opacity= .6
    ),
    name= ['unbiased','biased','corrected'][i] + ' r: {}'.format(round(pearsons[i],3))
    ) for i in [0,1,2]
]

layout = go.Layout(
    title= 'PCA distances against fst across sampling scenarios; VarEXC; Unscaled; Npops= {}'.format(N_pops),
    yaxis=dict(
        title='feature space distances'),
    xaxis=dict(
        title='Fst')
)

fig= go.Figure(data=fig_data, layout=layout)
iplot(fig)

**Fig. 9** Relationship between Fst and feature space distances across simulations and for the three scenarios considered: unbiased sampling, biased sampling and MS corrected biased sampling.

### Haplotype length and population number.

We began by conducting a test of the impact PCA on the relative distances of a single population genetic structure. Because a single scenario is not able to provide us with a statistical significance of the relation between genetic and PCA feature space distances, we then iterated that initial test across as many genetic structures as possible. This allowed us to extract an estimate of that relationship. We chose Pearson's *r* because of its apparent linearity.

However, in order to be able to compare different iterations we were limited to using the same haplotype length and population number. Going further, it is of interest to the practicioner to understand how this relation actually holds up across haplotype lengths and number of population numbers.


For this purpose we will package the block of script that performed the iterations into a function that takes as argument the number of populations and haplotype lengths.

This function is the core of the command-line ready script for this purpose. This script will take as arguments the total raange of population numbers and haplotype lengths to explore, and allow the user to select wether he wants his data to be scaled prior to feature reduction or not.

A [new directory]() was created to hold the final version of that product.

In [21]:

def controled_fsts(vector_lib,Eigen,length_haps,Scale,N_pops,n_comp,Iter,N_sims):

    ## Population Sizes and labels
    bias_scheme= np.random.choice(range(25,200),N_pops,replace= False)
    unbiased_sheme= np.repeat(N_sims,N_pops)

    bias_labels= np.repeat(np.array([x for x in range(N_pops)]),bias_scheme)
    unbias_labels= np.repeat(np.array([x for x in range(N_pops)]),unbiased_sheme)

    ### store distances between centroids
    biased_pairwise= []
    unbiased_pairwise= []
    corrected_pairwise= []

    ### store fsts
    fst_store= []

    ### store Pearson's r comparing gen_diffs and feature space diffs across scenarios
    biased_pears= []
    corrected_pears= []
    unbiased_pears= []

    ### triangular matrices extract.
    iu1= np.triu_indices(N_pops,1) # for centroid comparison

    iu_unbias= np.triu_indices(sum(unbiased_sheme),1)
    iu_bias= np.triu_indices(sum(bias_scheme),1)

    ### proceed.

    for rep in range(Iter):
        Pops= np.random.choice(vector_lib.shape[0],N_pops,replace= False)
        print('vectors selected: {}'.format(Pops))
        ########## FST

        freqs_selected= vector_lib[Pops,:length_haps]
        Pairwise= return_fsts2(freqs_selected)

        #fsts_compare = scale(Pairwise.fst)
        fsts_compare= Pairwise.fst
        fst_store.extend(fsts_compare)
        #########################################################
        ########### PCA ####################################
        #########################################################
        ############# unbiased sample

        #### generate data and perform PCA.
        data= []

        for k in range(N_pops):

            probs= vector_lib[Pops[k],:length_haps]

            m= unbiased_sheme[k]
            Haps= [[np.random.choice([1,0],p= [1-probs[x],probs[x]]) for x in range(length_haps)] for acc in range(m)]

            data.extend(Haps)

        data1= np.array(data)

        if Scale:
            data1= scale(data1)

        pca = PCA(n_components=n_comp, whiten=False,svd_solver='randomized').fit(data1)

        feat_unbias= pca.transform(data1)

        if Eigen:
            feat_unbias= feat_unbias * pca.explained_variance_ratio_

        ####### centroid comparison
        unbias_centroids= [np.mean(feat_unbias[[y for y in range(feat_unbias.shape[0]) if unbias_labels[y] == z],:],axis= 0) for z in range(N_pops)]
        unbias_centroids= np.array(unbias_centroids)

        unbias_pair_dist= pairwise_distances(unbias_centroids,metric= 'euclidean')
        unbias_pair_dist= unbias_pair_dist[iu1]

        #unbias_pair_dist= scale(unbias_pair_dist)
        unbiased_pairwise.extend(unbias_pair_dist)

        ######## ind distances
        ### genetic data
        unbias_gen_diffs= pairwise_distances(data1,metric= pairwise_gen)
        unbias_gen_diffs= np.array(unbias_gen_diffs)
        unbias_gen_diffs= unbias_gen_diffs[iu_unbias]

        ## feature space
        unbiased_feat_dist= pairwise_distances(feat_unbias, metric= 'euclidean')
        unbiased_feat_dist= unbiased_feat_dist[iu_unbias]
        #unbiased_feat_dist= scale(unbiased_feat_dist)

        unbiased_gen_pearson= pearsonr(unbiased_feat_dist,unbias_gen_diffs)

        unbiased_pears.append(unbiased_gen_pearson[0])

        #################################################
        ############## biased sample

        #### generate data and perform PCA
        data= []

        for k in range(N_pops):

            probs= vector_lib[Pops[k],:]

            m= bias_scheme[k]
            Haps= [[np.random.choice([1,0],p= [1-probs[x],probs[x]]) for x in range(length_haps)] for acc in range(m)]

            data.extend(Haps)

        data2= np.array(data)

        if Scale:
            data2= scale(data2)

        pca = PCA(n_components=n_comp, whiten=False,svd_solver='randomized').fit(data2)

        feat_bias= pca.transform(data2)

        if Eigen:
            feat_bias= feat_bias * pca.explained_variance_ratio_

        #### Centroid distances
        bias_centroids= [np.mean(feat_bias[[y for y in range(feat_bias.shape[0]) if bias_labels[y] == z],:],axis= 0) for z in range(N_pops)]
        bias_centroids= np.array(bias_centroids)

        bias_pair_dist= pairwise_distances(bias_centroids,metric= 'euclidean')
        bias_pair_dist= bias_pair_dist[iu1]
        #bias_pair_dist= scale(bias_pair_dist)
        biased_pairwise.extend(bias_pair_dist)

        ######## Ind distances
        ### genetic data
        bias_gen_diffs= pairwise_distances(data2,metric= pairwise_gen)
        bias_gen_diffs= np.array(bias_gen_diffs)
        bias_gen_diffs= bias_gen_diffs[iu_bias]

        ## feature space
        biased_feat_dist= pairwise_distances(feat_bias, metric= 'euclidean')
        biased_feat_dist= biased_feat_dist[iu_bias]
        #biased_feat_dist= scale(biased_feat_dist)

        biased_gen_pearson= pearsonr(biased_feat_dist,bias_gen_diffs)

        biased_pears.append(biased_gen_pearson[0])

        ###############################################################"
        ################## bias correct
        ### perform MS correction on biased samples
        feat_correct,var_comp= local_sampling_correct(data2,n_comp)

        ### centroid Distances
        centroids= [np.mean(feat_correct[[y for y in range(feat_correct.shape[0]) if bias_labels[y] == z],:],axis= 0) for z in range(N_pops)]
        centroids= np.array(centroids)
        pair_dist= pairwise_distances(centroids,metric= 'euclidean')
        pair_dist= pair_dist[iu1]
        #pair_dist= scale(pair_dist)
        corrected_pairwise.extend(pair_dist)

        ######## Ind distances

        ## feature space
        corrected_feat_dist= pairwise_distances(feat_correct, metric= 'euclidean')
        corrected_feat_dist= corrected_feat_dist[iu_bias]
        #corrected_feat_dist= scale(corrected_feat_dist)

        corr_gen_pearson= pearsonr(corrected_feat_dist,bias_gen_diffs)

        corrected_pears.append(corr_gen_pearson[0])


        t= np.array([
        fsts_compare,
        unbias_pair_dist,
        bias_pair_dist,
        pair_dist
        ]).T

    t= np.array([
        unbiased_pairwise,
        biased_pairwise,
        corrected_pairwise
    ]).T

    pearsons= [pearsonr(fst_store,t[:,x])[0] for x in range(t.shape[1])]

    return pearsons


